In [1]:
import pandas as pd
from textblob import TextBlob
import gensim
from gensim import corpora
from gensim.models import LdaModel
import pyLDAvis.gensim_models
import spacy
nlp = spacy.load('en_core_web_sm')

C:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\debug\cli\debugger_cli_common.py:19: DeprecationWarning: module 'sre_constants' is deprecated
  import sre_constants


In [2]:
def get_sentiment(review):
    testimonial = TextBlob(review)
    polarity = testimonial.sentiment.polarity
    # Classify the sentiment as positive, negative, or neutral based on the polarity score
    if polarity > 0:
        return 'positive'
    elif polarity < 0:
        return 'negative'
    else:
        return 'neutral'

In [3]:
def get_proccesed_doc(docs):
    doc = nlp(docs)
    doc1 = [token for token in doc if not token.is_stop]
    doc2 = [token.text for token in doc1 if token.is_alpha]
    final_text = ' '.join(doc2)
    return doc2

In [4]:
def get_proccesed_text(docs):
    doc = nlp(docs)
    doc1 = [token for token in doc if not token.is_stop]
    doc2 = [token.text for token in doc1 if token.is_alpha]
    final_text = ' '.join(doc2)
    return final_text

In [5]:
df  = pd.read_csv('final_data.csv')

In [6]:
df['Sentiment'] = df['Reviews'].apply(get_sentiment)

In [7]:
df.head()

,Ratings,Reviews,Sentiment
0,2,"Since the last 2-3 updates, the application wo...",positive
1,4,There's no feature to share my screen. Update ...,negative
2,2,"The app tries to improve, but takes a step bac...",positive
3,1,After the last update it's a nightmare! I am t...,negative
4,2,Switching phones is not easy with this app. I ...,negative


In [8]:
groups = df.groupby('Sentiment')

In [9]:
negative_comments = groups.get_group('negative')
positive_comments = groups.get_group('positive')

In [10]:
negative_comments.head()

,Ratings,Reviews,Sentiment
1,4,There's no feature to share my screen. Update ...,negative
3,1,After the last update it's a nightmare! I am t...,negative
4,2,Switching phones is not easy with this app. I ...,negative
7,1,Exceptionally horrible! Forces close. Freezes....,negative
11,1,It has been extremely buggy for the last few m...,negative


In [11]:
negative_comments_list = negative_comments['Reviews'].to_list()
positive_comments_list = positive_comments['Reviews'].to_list()

In [12]:
negative_document = list(map(get_proccesed_doc, negative_comments_list))
processed_negative_comments = list(map(get_proccesed_text, negative_comments_list))

In [13]:
positive_document = list(map(get_proccesed_doc, positive_comments_list))
processed_positive_comments = list(map(get_proccesed_text, positive_comments_list))

In [14]:
positive_dic = corpora.Dictionary(positive_document)
negative_dic = corpora.Dictionary(negative_document)

In [15]:
negative_corpus = [negative_dic.doc2bow(doc) for doc in negative_document]
positive_corpus = [positive_dic.doc2bow(doc) for doc in positive_document]

In [17]:
num_topic = 20
positive_model = LdaModel(positive_corpus, num_topics=num_topic, id2word=positive_dic, passes=25)
negative_model = LdaModel(negative_corpus, num_topics=num_topic, id2word=negative_dic, passes=25)

In [18]:
# Print the top 10 topics and their associated words
top_topics = positive_model.print_topics(num_topics=10, num_words=10)

for topic in top_topics:
    print(topic)

(6, '0.070*"chat" + 0.060*"group" + 0.050*"chats" + 0.045*"feature" + 0.027*"groups" + 0.024*"lock" + 0.023*"option" + 0.022*"messages" + 0.018*"like" + 0.016*"app"')
(13, '0.017*"privacy" + 0.013*"access" + 0.013*"successful" + 0.012*"mistake" + 0.009*"tiring" + 0.008*"app" + 0.008*"specific" + 0.008*"FB" + 0.008*"looks" + 0.007*"statuses"')
(5, '0.043*"app" + 0.039*"WhatsApp" + 0.030*"new" + 0.024*"features" + 0.023*"update" + 0.022*"channel" + 0.022*"like" + 0.017*"channels" + 0.016*"feature" + 0.014*"friends"')
(16, '0.052*"like" + 0.014*"add" + 0.013*"message" + 0.013*"send" + 0.013*"whatsapp" + 0.012*"want" + 0.010*"messages" + 0.010*"read" + 0.009*"check" + 0.008*"function"')
(7, '0.027*"new" + 0.020*"emoji" + 0.016*"update" + 0.016*"app" + 0.015*"old" + 0.015*"keyboard" + 0.014*"text" + 0.012*"like" + 0.010*"version" + 0.010*"chat"')
(10, '0.069*"video" + 0.056*"voice" + 0.035*"screen" + 0.027*"calls" + 0.025*"audio" + 0.017*"note" + 0.014*"sharing" + 0.011*"problem" + 0.010*"c

In [19]:
# Print the top 10 topics and their associated words
top_topics = negative_model.print_topics(num_topics=10, num_words=10)

for topic in top_topics:
    print(topic)

(9, '0.043*"apps" + 0.037*"issues" + 0.032*"screen" + 0.031*"connection" + 0.028*"calls" + 0.028*"try" + 0.025*"video" + 0.025*"phone" + 0.025*"WhatsApp" + 0.024*"update"')
(18, '0.040*"going" + 0.040*"chats" + 0.039*"typing" + 0.039*"like" + 0.038*"app" + 0.037*"page" + 0.036*"long" + 0.033*"search" + 0.031*"results" + 0.029*"group"')
(19, '0.075*"horrible" + 0.050*"stuffs" + 0.050*"skipping" + 0.049*"happend" + 0.017*"things" + 0.010*"day" + 0.010*"n" + 0.008*"gif" + 0.008*"Bring" + 0.008*"u"')
(17, '0.052*"account" + 0.047*"WhatsApp" + 0.042*"number" + 0.038*"issue" + 0.027*"app" + 0.025*"whatsapp" + 0.022*"new" + 0.021*"blocked" + 0.018*"time" + 0.017*"showed"')
(15, '0.086*"WhatsApp" + 0.056*"channels" + 0.051*"like" + 0.036*"worst" + 0.034*"new" + 0.034*"update" + 0.030*"app" + 0.028*"seriously" + 0.028*"confused" + 0.022*"version"')
(3, '0.097*"quality" + 0.056*"video" + 0.056*"status" + 0.046*"sent" + 0.041*"videos" + 0.038*"files" + 0.037*"needs" + 0.033*"upload" + 0.030*"bad"

In [34]:
import pyLDAvis.gensim

In [35]:
import joblib

In [42]:
pyLDAvis.enable_notebook()
positive_lda_visualization = pyLDAvis.gensim.prepare(positive_model, positive_corpus, positive_dic)
negative_lda_visualization = pyLDAvis.gensim.prepare(negative_model, negative_corpus, negative_dic)

In [43]:
positive_lda_visualization

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.106904  0.130399       1        1  14.200594
9      0.173768  0.046448       2        1  10.135554
14     0.151181  0.002763       3        1   9.240375
5      0.167683 -0.080936       4        1   8.959588
7      0.016624 -0.122007       5        1   5.949651
6      0.143466 -0.076775       6        1   5.352252
17     0.051823  0.143353       7        1   5.294374
3      0.012891  0.082991       8        1   4.790461
18     0.102566  0.014721       9        1   4.635628
12    -0.069414 -0.229893      10        1   4.605190
16     0.060989 -0.036494      11        1   4.297979
10    -0.001493  0.071860      12        1   4.114941
15    -0.020287 -0.223495      13        1   4.057877
19    -0.010681  0.017739      14        1   3.947273
2     -0.058291  0.075292      15        1   3.110046
8     -0.030003  0.096375      16        1   2.712774
4     -0.190268  0.077404      17        1   1.238450
1     -0.214256 -0.010105      18        1   1.223784
11    -0.210329  0.008248      19        1   1.075409
13    -0.182876  0.012113      20        1   1.057798, topic_info=        Term         Freq        Total Category  logprob  loglift
269   status  1054.000000  1054.000000  Default  30.0000  30.0000
324    video   630.000000   630.000000  Default  29.0000  29.0000
552  quality   398.000000   398.000000  Default  28.0000  28.0000
52    update   940.000000   940.000000  Default  27.0000  27.0000
378     chat   423.000000   423.000000  Default  26.0000  26.0000
..       ...          ...          ...      ...      ...      ...
146       nt     4.426416   143.637543  Topic20  -5.1911   1.0693
131   change     4.325822   191.977060  Topic20  -5.2141   0.7562
923   posted     3.499798    19.704529  Topic20  -5.4260   2.8208
32    delete     3.656275   178.824754  Topic20  -5.3822   0.6590
127     Good     3.557779    71.406904  Topic20  -5.4096   1.5497

[1310 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
5751     17  0.791967  ACCOUNT
3746     17  0.753848       AI
4604      7  0.086251      APP
4604     11  0.776257      APP
1986     12  0.892144  Ability
...     ...       ...      ...
155      13  0.009111    years
155      14  0.036446    years
155      15  0.091114    years
155      16  0.027334    years
155      20  0.027334    years

[3202 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 10, 15, 6, 8, 7, 18, 4, 19, 13, 17, 11, 16, 20, 3, 9, 5, 2, 12, 14])

In [44]:
negative_lda_visualization

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.117508  0.098436       1        1  19.496237
17     0.102727 -0.024761       2        1  12.445586
10    -0.029599  0.066164       3        1   8.022909
6      0.107708  0.010175       4        1   8.006998
9     -0.012085 -0.043130       5        1   6.145194
12     0.009628  0.114817       6        1   4.867964
13     0.169788 -0.001925       7        1   4.355230
3     -0.113735  0.294525       8        1   4.173653
1      0.040459  0.239798       9        1   4.039745
7      0.233483 -0.084731      10        1   3.749754
14    -0.075493 -0.133129      11        1   3.641052
15     0.198915 -0.081670      12        1   3.241476
11    -0.106113 -0.045573      13        1   3.211210
18     0.106194 -0.057542      14        1   2.764827
5     -0.049019 -0.076981      15        1   2.665969
0     -0.152529 -0.031402      16        1   2.471599
4     -0.088072 -0.063058      17        1   1.977878
16    -0.134890 -0.058215      18        1   1.970428
19    -0.167103 -0.043085      19        1   1.459687
2     -0.157774 -0.078712      20        1   1.292605, topic_info=          Term        Freq       Total Category  logprob  loglift
277     status  570.000000  570.000000  Default  30.0000  30.0000
51      update  730.000000  730.000000  Default  29.0000  29.0000
109   WhatsApp  795.000000  795.000000  Default  28.0000  28.0000
37         new  594.000000  594.000000  Default  27.0000  27.0000
208    quality  273.000000  273.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
816       told    1.096603    9.192708  Topic20  -6.2210   2.2223
663       turn    1.097726   33.151128  Topic20  -6.2200   0.9407
2023     means    0.805867    5.921549  Topic20  -6.5291   2.3541
626   problems    0.874203   18.620665  Topic20  -6.4477   1.2898
1294   display    0.694251   11.410585  Topic20  -6.6782   1.5490

[1004 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
1910     20  0.957247        Adding
164       8  0.671450  Additionally
165       5  0.249321       Android
165       9  0.598370       Android
165      13  0.099728       Android
...     ...       ...           ...
553      13  0.029110         wrong
553      16  0.043666         wrong
2478      4  0.182890     yesterday
2478     15  0.548669     yesterday
790      20  0.522928       youtube

[1483 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 18, 11, 7, 10, 13, 14, 4, 2, 8, 15, 16, 12, 19, 6, 1, 5, 17, 20, 3])

In [47]:
import re

# Initialize an empty set to store unique words
all_words = set()

# Extract words from each topic string and add them to the set
for topic in top_topics:
    words_with_weights = re.findall(r'"(.*?)"', topic[1])  # Extract words within double quotes
    for word in words_with_weights:
        all_words.add(word)

# Display all the unique words
print(all_words)


{'contacts', 'worst', 'u', 'load', 'typing', 'quality', 'star', 'weeks', 'bad', 'help', 'WhatsApp', 'going', 'blocked', 'day', 'apps', 'issues', 'page', 'lot', 'upload', 'suddenly', 'things', 'needs', 'screen', 'time', 'showed', 'status', 'happend', 'loading', 'stickers', 'Hated', 'account', 'group', 'experience', 'confused', 'try', 'phone', 'end', 'information', 'search', 'files', 'lost', 'chats', 'results', 'gif', 'code', 'emojis', 'like', 'n', 'access', 'sticker', 'keyboard', 'makes', 'number', 'forever', 'skipping', 'single', 'verification', 'channels', 'annoying', 'Bring', 'videos', 'horrible', 'connection', 'picture', 'update', 'calls', 'want', 'stuffs', 'chat', 'whatsapp', 'emoji', 'app', 'profile', 'seriously', 'frustrating', 'long', 'version', 'video', 'issue', 'new', 'fine', 'sent', 'taking'}


In [45]:
top_topics

[(9,
  '0.043*"apps" + 0.037*"issues" + 0.032*"screen" + 0.031*"connection" + 0.028*"calls" + 0.028*"try" + 0.025*"video" + 0.025*"phone" + 0.025*"WhatsApp" + 0.024*"update"'),
 (18,
  '0.040*"going" + 0.040*"chats" + 0.039*"typing" + 0.039*"like" + 0.038*"app" + 0.037*"page" + 0.036*"long" + 0.033*"search" + 0.031*"results" + 0.029*"group"'),
 (19,
  '0.075*"horrible" + 0.050*"stuffs" + 0.050*"skipping" + 0.049*"happend" + 0.017*"things" + 0.010*"day" + 0.010*"n" + 0.008*"gif" + 0.008*"Bring" + 0.008*"u"'),
 (17,
  '0.052*"account" + 0.047*"WhatsApp" + 0.042*"number" + 0.038*"issue" + 0.027*"app" + 0.025*"whatsapp" + 0.022*"new" + 0.021*"blocked" + 0.018*"time" + 0.017*"showed"'),
 (15,
  '0.086*"WhatsApp" + 0.056*"channels" + 0.051*"like" + 0.036*"worst" + 0.034*"new" + 0.034*"update" + 0.030*"app" + 0.028*"seriously" + 0.028*"confused" + 0.022*"version"'),
 (3,
  '0.097*"quality" + 0.056*"video" + 0.056*"status" + 0.046*"sent" + 0.041*"videos" + 0.038*"files" + 0.037*"needs" + 0.033